<a href="https://colab.research.google.com/github/Base-ai/free-indextts-1.5-on-colab/blob/main/index-tts-2-on-colab-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 安装工具链
%pip install uv
!uv tool install "huggingface_hub[cli]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 120.9 MB/s eta 0:00:00
Resolved 17 packages in 102ms
Prepared 17 packages in 216ms
Installed 17 packages in 30ms
 + certifi==2025.8.3
 + charset-normalizer==3.4.3
 + filelock==3.19.1
 + fsspec==2025.9.0
 + hf-xet==1.1.10
 + huggingface-hub==0.35.3
 + idna==3.10
 + inquirerpy==0.3.4
 + packaging==25.0
 + pfzy==0.3.4
 + prompt-toolkit==3.0.52
 + pyyaml==6.0.3
 + requests==2.32.5
 + tqdm==4.67.1
 + typing-extensions==4.15.0
 + urllib3==2.5.0
 + wcwidth==0.2.14
Installed 3 executables: hf, huggingface-cli, tiny-agents


In [2]:
# ✅ Step 1: 获取最近n个提交
import requests
import ipywidgets as widgets
from IPython.display import display
import subprocess

# 仓库信息
repo = "index-tts/index-tts"
commits_url = f"https://api.github.com/repos/{repo}/commits"
response = requests.get(commits_url)
commits = response.json()

# 提取前两个提交 SHA 和信息
options = []
for commit in commits[:3]:
    sha = commit["sha"]
    message = commit["commit"]["message"]
    options.append((f"{message[:60]}... ({sha[:7]})", sha))  # 显示前60字+SHA前7位

# ✅ Step 2: 创建下拉菜单和按钮
dropdown = widgets.Dropdown(
    options=options,
    description='选择版本:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

button = widgets.Button(description="下载选定版本", button_style='success')
output = widgets.Output()

# ✅ Step 3: 下载并解压所选版本
def download_selected_version(sha):
    url = f"https://github.com/index-tts/index-tts/archive/{sha}.zip"
    subprocess.run(["wget", "-O", "main.zip", url])
    subprocess.run(["unzip", "main.zip"])
    subprocess.run(["mv", f"index-tts-{sha}", "index-tts"])
    subprocess.run(["rm", "main.zip"])

def on_button_clicked(b):
    with output:
        output.clear_output()
        print(f"正在下载版本：{dropdown.label}")
        download_selected_version(dropdown.value)
        print("✅ 下载完成，已解压到 index-tts 文件夹")

button.on_click(on_button_clicked)

# ✅ Step 4: 显示控件
display(dropdown, button, output)


Dropdown(description='选择版本:', layout=Layout(width='80%'), options=(('Simple streaming return implementation, l…

Button(button_style='success', description='下载选定版本', style=ButtonStyle())

Output()

In [3]:
# 安装依赖
!cd /content/index-tts/ && uv sync --all-extras

/bin/bash: line 1: cd: /content/index-tts/: No such file or directory


In [4]:
# 下载模型 checkpoints
!hf download IndexTeam/IndexTTS-2 --local-dir=/content/index-tts/checkpoints

Fetching 20 files:   0% 0/20 [00:00<?, ?it/s]Downloading 'feat1.pt' to '/content/index-tts/checkpoints/.cache/huggingface/download/MKSxOKdeAa3yNVriUN75BCpdwis=.f219cb447d80216ba615666da2ff8d63ac544eee26657f3a7b278692bf7a67c4.incomplete'

config.yaml: 2.88kB [00:00, 11.2MB/s]
Download complete. Moving file to /content/index-tts/checkpoints/config.yaml

README.md: 2.50kB [00:00, 15.1MB/s]
Download complete. Moving file to /content/index-tts/checkpoints/README.md

Modelfile: 100% 360/360 [00:00<00:00, 2.97MB/s]
Download complete. Moving file to /content/index-tts/checkpoints/qwen0.6bemo4-merge/Modelfile

.gitattributes: 1.73kB [00:00, 12.3MB/s]
Download complete. Moving file to /content/index-tts/checkpoints/.gitattributes
Fetching 20 files:   5% 1/20 [00:00<00:06,  2.83it/s]
gpt.pth:   0% 0.00/3.48G [00:00<?, ?B/s]

bpe.model:   0% 0.00/476k [00:00<?, ?B/s]


feat2.pt:   0% 0.00/375k [00:00<?, ?B/s]



feat1.pt:   0% 0.00/57.2k [00:00<?, ?B/s]Downloading 'qwen0.6bemo4-merge/added_tokens.

In [5]:
# 为了直接利用 Gradio 的内网穿透功能
# 需要对 index—tts/webui.py 进行修改

# 如果因为 IndexTTS 官方 GitHub 仓库的更改造成本代码不可用
# 还可以手动修改 webui.py ，在 launch 函数调用中加上 `share = True`


%cd /content/index-tts

launch_line = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port)'
launch_line_with_share = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port, share=True)'

with open("webui.py", encoding='utf8') as f:
  webui_with_share = []
  for line in f.readlines():
    if line.strip() == launch_line:
      webui_with_share.append(line.replace(launch_line, launch_line_with_share))
    else:
      webui_with_share.append(line)

  with open("webui_with_share.py", 'w', encoding='utf8') as f:
    f.writelines(webui_with_share)

/content/index-tts


FileNotFoundError: [Errno 2] No such file or directory: 'webui.py'

In [ ]:
# 启动

%cd /content/index-tts/
!uv run webui_with_share.py